In [139]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler


In [140]:
# loading pickle file for standardization
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [141]:
#Loading ANN model 
from keras.models import load_model
import keras.metrics

# Load your model and specify the custom objects
model = load_model('temp1.h5', custom_objects={'mse': keras.metrics.mean_squared_error})


In [142]:
scaler

MinMaxScaler()

In [143]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,404 (9.39 KB)

 Trainable params: 2,402 (9.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [150]:
#Reading Validation dataset
data = pd.read_csv('Validation Dataset.csv')

In [151]:
data.head()

,SI,Date,Unnamed: 2,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),rainfall (mm),Air Temp,Air Temp C
0,1,05-01-2019,NaN,8.43,8.24,8.00,7.60,1.778,48.8,9.333
1,2,05-02-2019,NaN,8.95,8.36,8.01,7.60,4.826,53.7,12.056
2,3,05-03-2019,NaN,10.19,8.95,8.22,7.65,3.810,55.6,13.111
3,4,05-04-2019,NaN,11.24,9.67,8.62,7.78,8.128,53.3,11.833
4,5,05-05-2019,NaN,12.02,10.34,9.09,7.98,0.000,52.3,11.278


##### Feature Engineering

In [152]:
data.drop(columns=['SI','Unnamed: 2','Air Temp'],axis=1,inplace=True)

In [153]:
data.head()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),rainfall (mm),Air Temp C
0,05-01-2019,8.43,8.24,8.00,7.60,1.778,9.333
1,05-02-2019,8.95,8.36,8.01,7.60,4.826,12.056
2,05-03-2019,10.19,8.95,8.22,7.65,3.810,13.111
3,05-04-2019,11.24,9.67,8.62,7.78,8.128,11.833
4,05-05-2019,12.02,10.34,9.09,7.98,0.000,11.278


In [154]:
# Convert 'Date' to datetime 
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create the 'day_of_year' column
data['day_of_year'] = data['Date'].dt.dayofyear


In [155]:
data.head()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),rainfall (mm),Air Temp C,day_of_year
0,2019-05-01,8.43,8.24,8.00,7.60,1.778,9.333,121.0
1,2019-05-02,8.95,8.36,8.01,7.60,4.826,12.056,122.0
2,2019-05-03,10.19,8.95,8.22,7.65,3.810,13.111,123.0
3,2019-05-04,11.24,9.67,8.62,7.78,8.128,11.833,124.0
4,2019-05-05,12.02,10.34,9.09,7.98,0.000,11.278,125.0


In [156]:
data['Rainfall(inch)'] = data['rainfall (mm)'] * 0.0393701

In [157]:
data.head()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),rainfall (mm),Air Temp C,day_of_year,Rainfall(inch)
0,2019-05-01,8.43,8.24,8.00,7.60,1.778,9.333,121.0,0.07
1,2019-05-02,8.95,8.36,8.01,7.60,4.826,12.056,122.0,0.19
2,2019-05-03,10.19,8.95,8.22,7.65,3.810,13.111,123.0,0.15
3,2019-05-04,11.24,9.67,8.62,7.78,8.128,11.833,124.0,0.32
4,2019-05-05,12.02,10.34,9.09,7.98,0.000,11.278,125.0,0.00


In [158]:
data.drop(columns=['rainfall (mm)'],axis=1,inplace=True)

In [159]:
data.head()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),Air Temp C,day_of_year,Rainfall(inch)
0,2019-05-01,8.43,8.24,8.00,7.60,9.333,121.0,0.07
1,2019-05-02,8.95,8.36,8.01,7.60,12.056,122.0,0.19
2,2019-05-03,10.19,8.95,8.22,7.65,13.111,123.0,0.15
3,2019-05-04,11.24,9.67,8.62,7.78,11.833,124.0,0.32
4,2019-05-05,12.02,10.34,9.09,7.98,11.278,125.0,0.00


In [160]:
X = data[['Air Temp C','Rainfall(inch)','day_of_year']]

X.head()

,Air Temp C,Rainfall(inch),day_of_year
0,9.333,0.07,121.0
1,12.056,0.19,122.0
2,13.111,0.15,123.0
3,11.833,0.32,124.0
4,11.278,0.00,125.0


In [161]:
#Standardization

validation_data_scaled = scaler.transform(X)

In [162]:
#prediction

predicted_values = model.predict(validation_data_scaled)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [163]:
predicted_values

array([[18.700241 , 21.672953 ],
       [19.20749  , 21.692139 ],
       [19.398432 , 21.701345 ],
       ...,
       [ 1.4292933,  2.424792 ],
       [ 1.4292932,  2.4247923],
       [ 1.4292932,  2.4247923]], dtype=float32)

In [164]:
#creating dataframe for predicted values
df = pd.DataFrame(predicted_values, columns=['Temp_Pred(0.9m)','Temp_Pred(1.8m)'])

In [165]:
df.head()

,Temp_Pred(0.9m),Temp_Pred(1.8m)
0,18.700241,21.672953
1,19.207491,21.692139
2,19.398432,21.701345
3,19.406530,21.872974
4,19.221674,21.823034


In [166]:
#concate both dataframes

X = pd.concat([X, df], axis=1)

In [167]:
X.head()

,Air Temp C,Rainfall(inch),day_of_year,Temp_Pred(0.9m),Temp_Pred(1.8m)
0,9.333,0.07,121.0,18.700241,21.672953
1,12.056,0.19,122.0,19.207491,21.692139
2,13.111,0.15,123.0,19.398432,21.701345
3,11.833,0.32,124.0,19.406530,21.872974
4,11.278,0.00,125.0,19.221674,21.823034


In [ ]:

# Convert temperatures to equivalent at 1ft
data["Temp_pred_1ft"] = X["Temp_Pred(0.9m)"] / (0.9 * 3.28084 / 1) 


In [175]:
# Convert predictions from meters to feet
# meters_to_feet = 3.28084
# predictions_feet = data['Temperature3ft (0.91m)'] * meters_to_feet

In [176]:
data.head()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),Air Temp C,day_of_year,Rainfall(inch),Temp_pred_1ft
0,2019-05-01,8.43,8.24,8.00,7.60,9.333,121.0,0.07,6.333148
1,2019-05-02,8.95,8.36,8.01,7.60,12.056,122.0,0.19,6.504937
2,2019-05-03,10.19,8.95,8.22,7.65,13.111,123.0,0.15,6.569602
3,2019-05-04,11.24,9.67,8.62,7.78,11.833,124.0,0.32,6.572345
4,2019-05-05,12.02,10.34,9.09,7.98,11.278,125.0,0.00,6.509740


In [177]:
data.head()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),Air Temp C,day_of_year,Rainfall(inch),Temp_pred_1ft
0,2019-05-01,8.43,8.24,8.00,7.60,9.333,121.0,0.07,6.333148
1,2019-05-02,8.95,8.36,8.01,7.60,12.056,122.0,0.19,6.504937
2,2019-05-03,10.19,8.95,8.22,7.65,13.111,123.0,0.15,6.569602
3,2019-05-04,11.24,9.67,8.62,7.78,11.833,124.0,0.32,6.572345
4,2019-05-05,12.02,10.34,9.09,7.98,11.278,125.0,0.00,6.509740


In [178]:
data.tail()

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),Air Temp C,day_of_year,Rainfall(inch),Temp_pred_1ft
1091,NaT,9.6958,7.8833,6.8750,6.2292,11.389,NaN,0.00,0.484054
1092,NaT,9.0875,8.2375,7.3708,6.4833,4.000,NaN,0.05,0.484054
1093,NaT,7.5333,7.7667,7.4625,6.7125,4.500,NaN,0.01,0.484054
1094,NaT,6.9167,7.2375,7.2583,6.8000,5.611,NaN,0.00,0.484054
1095,NaT,7.1583,7.1042,7.1000,6.8000,7.667,NaN,0.00,0.484054


In [179]:
data

,Date,Temperature_1ft (0.3m),Temperature2ft (0.6m),Temperature3ft (0.91m),Temperature4ft (1.21m),Air Temp C,day_of_year,Rainfall(inch),Temp_pred_1ft
0,2019-05-01,8.4300,8.2400,8.0000,7.6000,9.333,121.0,0.07,6.333148
1,2019-05-02,8.9500,8.3600,8.0100,7.6000,12.056,122.0,0.19,6.504937
2,2019-05-03,10.1900,8.9500,8.2200,7.6500,13.111,123.0,0.15,6.569602
3,2019-05-04,11.2400,9.6700,8.6200,7.7800,11.833,124.0,0.32,6.572345
4,2019-05-05,12.0200,10.3400,9.0900,7.9800,11.278,125.0,0.00,6.509740
...,...,...,...,...,...,...,...,...,...
1091,NaT,9.6958,7.8833,6.8750,6.2292,11.389,NaN,0.00,0.484054
1092,NaT,9.0875,8.2375,7.3708,6.4833,4.000,NaN,0.05,0.484054
1093,NaT,7.5333,7.7667,7.4625,6.7125,4.500,NaN,0.01,0.484054
1094,NaT,6.9167,7.2375,7.2583,6.8000,5.611,NaN,0.00,0.484054


In [ ]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

# Paths to your files
model_path = 'temp.h5'  # Replace with the path to your .h5 model
scaler_path = 'scaler.pkl'  # Replace with the path to your scaler file
validation_data_path = 'Validate_Data.csv'  # Replace with the path to your validation data

# Step 1: Load the model
print("Loading model...")
model = load_model(model_path)

# Step 2: Load the scaler
print("Loading scaler...")
with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)

# Step 3: Load and preprocess the validation data
print("Loading validation data...")
validation_data = pd.read_csv(validation_data_path)

# Extract features (X) and true labels (y) if applicable
X_val = validation_data.iloc[:, :-1]  # Assuming all but the last column are features
y_true_mm = validation_data.iloc[:, -1]  # Assuming the last column is the true target in mm

# Scale the validation features using the loaded scaler
print("Scaling validation features...")
X_val_scaled = scaler.transform(X_val)

# Step 4: Make predictions
print("Making predictions...")
y_pred_mm = model.predict(X_val_scaled)

# Step 5: Convert predictions from mm to feet
print("Converting predictions to feet...")
mm_to_feet = 0.00328084
y_pred_ft = y_pred_mm * mm_to_feet

# Step 6: Validation and visualization
# Combine true and predicted values for comparison
comparison = pd.DataFrame({
    'True_Temperature_mm': y_true_mm,
    'Predicted_Temperature_mm': y_pred_mm.flatten(),
    'Predicted_Temperature_ft': y_pred_ft.flatten()
})

# Save the results to a CSV for further analysis
output_path = 'Predicted_Results.csv'
comparison.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")

# Optional: Visualize the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(y_true_mm, label='True Temperature (mm)', linestyle='dotted', marker='o')
plt.plot(y_pred_mm.flatten(), label='Predicted Temperature (mm)', linestyle='solid', marker='x')
plt.plot(y_pred_ft.flatten(), label='Predicted Temperature (ft)', linestyle='dashed', marker='s')
plt.title('Temperature Predictions')
plt.xlabel('Sample Index')
plt.ylabel('Temperature')
plt.legend()
plt.grid()
plt.show()
